# TCN Architecture Analysis (Execution-Ready, Updated 2026-02-18)

This notebook is aligned with the current TAPE environment updates:
- drawdown lambda carry-over + decay
- rebalanced penalties with penalty budget cap
- intra-step TAPE delta shaping (rolling potential difference)
- step-level Sharpe checkpointing
- updated episode-length curriculum schedule


## 1) Setup and Imports


In [1]:
import os, sys, socket, pathlib
print("cwd:", pathlib.Path.cwd())
print("python:", sys.executable)
print("hostname:", socket.gethostname())
print("has /content:", os.path.exists("/content"))
print("has /mnt/c:", os.path.exists("/mnt/c"))

cwd: /content
python: /usr/bin/python3
hostname: 01c598c115af
has /content: True
has /mnt/c: False


In [2]:
%cd /content/adaptive_portfolio_rl   # or /content/drive/MyDrive/adaptive_portfolio_rl
!git status
!git add src/environment_tape_rl.py src/config.py src/train_rl.py src/notebook_helpers/tcn_phase1.py
!git commit -m "Reward shaping: signed terminal TAPE + neutral band + Gate A wiring/logging"
!git push origin main

[Errno 2] No such file or directory: '/content/adaptive_portfolio_rl # or /content/drive/MyDrive/adaptive_portfolio_rl'
/content
fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git


In [ ]:
#%cd /content/adaptive_portfolio_rl
#!ls -la data
#!find . -type d -name "__pycache__" | head

In [ ]:
#%cd /content/adaptive_portfolio_rl
#!rm -f data/daily_ohlcv_assets.csv
#!rm -f data/master_features_NORMALIZED.csv
#!rm -f data/processed_daily_macro_features.csv
#!find . -type d -name "__pycache__" -prune -exec rm -rf {} +
#!ls -la data

In [ ]:
# Cell A: sync repo
import os
if not os.path.exists("/content/adaptive_portfolio_rl/.git"):
    !git clone https://github.com/Dave-DKings/tape_tcn_project.git /content/adaptive_portfolio_rl
%cd /content/adaptive_portfolio_rl
!git fetch origin
!git reset --hard origin/main

In [ ]:
# Cell B: install project requirements (Colab-safe pinned stack)
#%pip install -q -r requirements.txt


In [ ]:
#%pip install -q jedi==0.19.2

In [ ]:
import numpy as np, pandas as pd, tensorflow as tf
import pandas_ta_classic as ta_classic
print('Versions:', np.__version__, pd.__version__, tf.__version__)
print('TF GPUs visible:', tf.config.list_physical_devices('GPU'))
!pip check


In [ ]:
# ============================================================================
# SETUP: PROJECT ROOT, IMPORTS, REPRODUCIBILITY
# ============================================================================
import os
import sys
import json
import warnings
import importlib
from copy import deepcopy
from pathlib import Path
from datetime import datetime

# Resolve project root robustly
project_root = Path.cwd()
if project_root.name != 'adaptive_portfolio_rl':
    if (project_root / 'adaptive_portfolio_rl').exists():
        project_root = project_root / 'adaptive_portfolio_rl'
    elif (project_root.parent / 'adaptive_portfolio_rl').exists():
        project_root = project_root.parent / 'adaptive_portfolio_rl'

# Ensure imports resolve to this project only
sys.path.insert(0, str(project_root))
sys.path.insert(0, str(project_root / 'src'))

# Scientific stack
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import tensorflow as tf

# IMPORTANT: force-reload local modules so config edits take effect without stale state
import src.config as config_module
import src.data_utils as data_utils_module
import src.notebook_helpers.tcn_phase1 as tcn_phase1_module
importlib.reload(config_module)
importlib.reload(data_utils_module)
importlib.reload(tcn_phase1_module)

# Project imports (from freshly reloaded modules)
from src.data_utils import DataProcessor
from src.config import get_active_config, PROFILE_BALANCED_GROWTH, ASSET_TICKERS
from src.reproducibility_helper import set_all_seeds
from src.csv_logger import CSVLogger
from src.notebook_helpers.tcn_phase1 import (
    identify_covariance_columns,
    Phase1Dataset,
    run_experiment6_tape,
    evaluate_experiment6_checkpoint,
    create_experiment6_result_stub,
    load_training_metadata_into_config,
)

warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

RANDOM_SEED = 42
set_all_seeds(RANDOM_SEED, deterministic=True)

import logging
logging.getLogger("src.environment_tape_rl").setLevel(logging.WARNING)

gpus = tf.config.list_physical_devices("GPU")
print("TF GPUs:", gpus)
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)

print('✅ Setup complete')
print('Project root:', project_root)
print('Config module path:', config_module.__file__)
print('Active fetch range from module:', config_module.DATA_FETCH_START_DATE, '->', config_module.DATA_FETCH_END_DATE)
print('Active analysis range from module:', config_module.ANALYSIS_START_DATE, '->', config_module.ANALYSIS_END_DATE)
print('Train split end from module:', config_module.TRAIN_TEST_SPLIT_DATE)
print('TensorFlow:', tf.__version__)
print('NumPy:', np.__version__)
print('Pandas:', pd.__version__)



## 2) Config and Run Controls


In [ ]:
# ============================================================================
# LOAD ACTIVE CONFIG + CURRENT CONTROL SNAPSHOT
# ============================================================================
config = get_active_config('phase1')

# Force a single source-of-truth date window for this notebook run
# (keeps paper training split, extends test window to ~3 years)
config['DATA_FETCH_START_DATE'] = '2003-09-02'
config['DATA_FETCH_END_DATE'] = '2024-09-01'
config['ANALYSIS_START_DATE'] = '2003-09-02'
config['ANALYSIS_END_DATE'] = '2024-09-01'
config['TRAIN_TEST_SPLIT_DATE'] = '2021-09-01'

print('Forced notebook date window:', config['DATA_FETCH_START_DATE'], '->', config['DATA_FETCH_END_DATE'])
print('Forced analysis window:', config['ANALYSIS_START_DATE'], '->', config['ANALYSIS_END_DATE'])
print('Forced train split end:', config['TRAIN_TEST_SPLIT_DATE'])

# Keep defaults from config unless explicitly changed below
config['agent_params']['actor_critic_type'] = 'TCN'
config['agent_params']['evaluation_mode'] = config['agent_params'].get('evaluation_mode', 'mode')
config['training_params']['update_log_interval'] = 1

ppo = config['agent_params'].get('ppo_params', {})
env = config.get('environment_params', {})
dd = env.get('drawdown_constraint', {})
tp = config.get('training_params', {})

print('CONFIG SNAPSHOT')
print('Phase:', config['phase_name'])
print('Tickers:', config['ASSET_TICKERS'])
print('Date range:', config['ANALYSIS_START_DATE'], '->', config['ANALYSIS_END_DATE'])
print('Architecture:', config['agent_params']['actor_critic_type'])
print('TCN filters:', config['agent_params'].get('tcn_filters'))
print('Dirichlet activation:', config['agent_params'].get('dirichlet_alpha_activation'))
print('PPO: epochs=', ppo.get('num_ppo_epochs'), 'clip=', ppo.get('policy_clip'), 'actor_lr=', ppo.get('actor_lr'), 'critic_lr=', ppo.get('critic_lr'), 'target_kl=', ppo.get('target_kl'))

print()
print('REWARD + RISK CONTROLS')
print('target_turnover=', env.get('target_turnover'), 'turnover_penalty_scalar=', env.get('turnover_penalty_scalar'))
print('concentration_penalty_scalar=', env.get('concentration_penalty_scalar'), 'top_weight_penalty_scalar=', env.get('top_weight_penalty_scalar'))
print('action_realization_penalty_scalar=', env.get('action_realization_penalty_scalar'))
print('penalty_budget_ratio=', env.get('penalty_budget_ratio'))
print('drawdown: penalty_coef=', dd.get('penalty_coef'), 'lambda_floor=', dd.get('lambda_floor'), 'lambda_carry_decay=', dd.get('lambda_carry_decay'))
print('drawdown: target=', dd.get('target'), 'tolerance=', dd.get('tolerance'), 'lambda_max=', dd.get('lambda_max'))

print()
print('INTRA-STEP TAPE DELTA')
print('enabled=', env.get('intra_step_tape_delta_enabled'), 'window=', env.get('intra_step_tape_delta_window'), 'min_history=', env.get('intra_step_tape_delta_min_history'))
print('beta=', env.get('intra_step_tape_delta_beta'), 'clip=', env.get('intra_step_tape_delta_clip'))

print()
print('CHECKPOINT RULES')
print('tape_checkpoint_threshold=', tp.get('tape_checkpoint_threshold'))
print('periodic_checkpoint_every_steps=', tp.get('periodic_checkpoint_every_steps'))
print('high_watermark_checkpoint_enabled=', tp.get('high_watermark_checkpoint_enabled'), 'threshold=', tp.get('high_watermark_sharpe_threshold'))
print('step_sharpe_checkpoint_enabled=', tp.get('step_sharpe_checkpoint_enabled'), 'threshold=', tp.get('step_sharpe_checkpoint_threshold'))

print()
print('CURRICULUM')
print('episode_length_curriculum_schedule=', tp.get('episode_length_curriculum_schedule'))
print('turnover_penalty_curriculum=', tp.get('turnover_penalty_curriculum'))


In [ ]:
# ============================================================================
# OPTIONAL OVERRIDES (SAFE, CURRENT-ALIGNED)
# ============================================================================
APPLY_OVERRIDES = False

if APPLY_OVERRIDES:
    # Compact run controls
    config['training_params']['max_total_timesteps'] = 30_000
    config['training_params']['timesteps_per_ppo_update'] = 504

    # PPO stability controls
    ppo = config['agent_params']['ppo_params']
    ppo.update({
        'policy_clip': 0.10,
        'num_ppo_epochs': 4,
        'actor_lr': 2e-5,
        'critic_lr': 3e-4,
        'target_kl': 0.015,
        'kl_stop_multiplier': 1.2,
        'minibatches_before_kl_stop': 1,
    })

    # Full-horizon episodes (disable episode-length curriculum)
    config['training_params']['use_episode_length_curriculum'] = False
    config['training_params']['episode_length_curriculum_schedule'] = [
        {'threshold': 0, 'limit': None},
    ]

    # Turnover / reward controls
    env = config['environment_params']
    env['target_turnover'] = 0.60
    env['turnover_target_band'] = 0.20
    env['turnover_penalty_scalar'] = 1.5
    env['dsr_scalar'] = 2.0
    config['training_params']['evaluation_turnover_penalty_scalar'] = 1.5

    # Penalty rebalancing
    env['concentration_penalty_scalar'] = 2.0
    env['concentration_target_hhi'] = 0.14
    env['top_weight_penalty_scalar'] = 1.5
    env['target_top_weight'] = 0.22
    env['action_realization_penalty_scalar'] = 0.5
    env['penalty_budget_ratio'] = 1.25

    # Drawdown controller (gentler early adaptation)
    dd = env['drawdown_constraint']
    dd.update({
        'enabled': True,
        'target': 0.18,
        'penalty_coef': 1.5,
        'dual_learning_rate': 0.10,
        'lambda_init': 0.50,
        'lambda_floor': 0.00,
        'lambda_max': 5.0,
        'lambda_carry_decay': 0.7,
        'tolerance': -0.015,
        'penalty_reference': 'trigger_boundary',
        'cooling_rate': 0.35,
    })

    # Intra-step TAPE delta shaping (reduced)
    env['intra_step_tape_delta_enabled'] = True
    env['intra_step_tape_delta_window'] = 60
    env['intra_step_tape_delta_min_history'] = 20
    env['intra_step_tape_delta_beta'] = 0.01
    env['intra_step_tape_delta_clip'] = 0.20

    # Step-level Sharpe checkpointing (save any step where Sharpe >= 0.5)
    config['training_params']['step_sharpe_checkpoint_enabled'] = True
    config['training_params']['step_sharpe_checkpoint_threshold'] = 0.5

    print('Overrides applied')
else:
    print('APPLY_OVERRIDES=False (using config defaults)')




In [ ]:
# ============================================================================
# VARIANT SETTINGS (TCN FAMILY)
# ============================================================================
VARIANT_SETTINGS = {
    'TCN': {
        'actor_critic_type': 'TCN',
        'use_attention': False,
        'use_fusion': False,
        'results_root': Path('tcn_results'),
    },
    'TCN_ATTENTION': {
        'actor_critic_type': 'TCN_ATTENTION',
        'use_attention': True,
        'use_fusion': False,
        'results_root': Path('tcn_att_results'),
    },
    'TCN_FUSION': {
        'actor_critic_type': 'TCN_FUSION',
        'use_attention': False,
        'use_fusion': True,
        'results_root': Path('tcn_fusion_results'),
    },
}

ACTIVE_VARIANT = 'TCN_FUSION'  # change to: TCN, TCN_ATTENTION, TCN_FUSION

if ACTIVE_VARIANT not in VARIANT_SETTINGS:
    raise ValueError(f'Unsupported ACTIVE_VARIANT: {ACTIVE_VARIANT}')

v = VARIANT_SETTINGS[ACTIVE_VARIANT]
config['agent_params']['actor_critic_type'] = v['actor_critic_type']
config['agent_params']['use_attention'] = v['use_attention']
config['agent_params']['use_fusion'] = v['use_fusion']

config['training_params']['use_episode_length_curriculum'] = False
config['training_params']['episode_length_curriculum_schedule'] = [
    {'threshold': 0, 'limit': None},
]


LATEST_VARIANT = ACTIVE_VARIANT
LATEST_RESULTS_ROOT = str(v['results_root'])

print('✅ Variant applied:', ACTIVE_VARIANT)
print('results_root:', LATEST_RESULTS_ROOT)


## 3) Data Pipeline (Features + Actuarial)


In [ ]:
config = get_active_config("phase1")

# Optional guard for legacy/invalid macro codes in old configs.
# Current config is already updated, so this is usually a no-op.
macro_cfg = config["feature_params"]["macro_data"]
invalid_legacy_codes = {"NAPM", "MOVEINDEX", "ISM/MAN_PMI", "MOVE"}
before = len(macro_cfg.get("fred_series_config", []))
macro_cfg["fred_series_config"] = [
    s for s in macro_cfg.get("fred_series_config", [])
    if s.get("code") not in invalid_legacy_codes
]
after = len(macro_cfg.get("fred_series_config", []))
print(f"FRED series config: {before} -> {after} (removed legacy invalid codes: {before - after})")

# Keep legacy engineered-column disables for backward-compatible runs.
legacy_macro_columns = {"ISM_MAN_PMI_level", "ISM_MAN_PMI_diff", "MOVE_level", "MOVE_zscore"}
disabled = set(config["feature_params"]["feature_selection"]["disabled_features"])
newly_added = sorted(list(legacy_macro_columns - disabled))
disabled.update(legacy_macro_columns)
config["feature_params"]["feature_selection"]["disabled_features"] = sorted(disabled)
if newly_added:
    print("Added legacy macro disables:", newly_added)
else:
    print("Legacy macro disable set already present")


In [ ]:
# ============================================================================
# DATA LOADING + FEATURE ENGINEERING
# ============================================================================
processor = DataProcessor(config)

print('=' * 80)
print('LOADING AND PROCESSING DATA')
print('=' * 80)

FORCE_DATA_REFRESH = False  # Set True once when you want to rebuild cache for new date bounds
raw_df = processor.load_ohlcv_data(
    start_date=config['DATA_FETCH_START_DATE'],
    end_date=config['DATA_FETCH_END_DATE'],
    force_download=FORCE_DATA_REFRESH,
)
print('Raw shape:', raw_df.shape)
print('Raw dates:', raw_df['Date'].min(), '→', raw_df['Date'].max())

# Core feature pipeline
df = processor.calculate_log_returns(raw_df, periods=[1, 5, 10, 21])
df = processor.calculate_return_statistics(df, window=21)
df = processor.calculate_technical_indicators(df)
df = processor.calculate_dynamic_covariance_features(df)
df = processor.add_regime_features(df)
df = processor.add_fundamental_features(df)

macro_cfg = config.get('feature_params', {}).get('macro_data')
if macro_cfg is not None:
    macro_df, macro_cols = processor._build_macro_feature_frame(macro_cfg, df['Date'].min(), df['Date'].max())
    if macro_df is not None and macro_cols:
        df = df.merge(macro_df, on='Date', how='left')
        print(f'Macro features added: {len(macro_cols)}')

df = processor.add_quant_alpha_features(df)
df = processor.add_cross_sectional_features(df)
df = processor.add_actuarial_features(df)

master_df = df.copy()
feature_cols = processor.get_feature_columns('phase1')
present_feature_cols = [c for c in feature_cols if c in master_df.columns]

print('Final master_df shape:', master_df.shape)
print('Expected feature cols:', len(feature_cols))
print('Present feature cols :', len(present_feature_cols))

nan_counts = master_df[present_feature_cols].isna().sum()
nan_cols = nan_counts[nan_counts > 0].sort_values(ascending=False)
print('Feature columns with NaN:', len(nan_cols))
if len(nan_cols) > 0:
    display(nan_cols.head(20))


In [ ]:
act_cols = [c for c in master_df.columns if c.startswith("Actuarial_")]
print("Actuarial columns present:", len(act_cols))
print(act_cols[:10])

if act_cols:
    print(master_df[act_cols].describe().T[["mean","std","min","max"]].head(20))

In [ ]:
# ============================================================================
# FIXED DATE SPLIT (TRAIN/TEST)
# ============================================================================
analysis_start = pd.Timestamp(config.get('ANALYSIS_START_DATE', '2003-09-02'))
analysis_end = pd.Timestamp(config.get('ANALYSIS_END_DATE', '2024-09-01'))

train_end_date = pd.Timestamp(config.get('TRAIN_TEST_SPLIT_DATE', '2021-09-01'))
test_start_date = train_end_date + pd.Timedelta(days=1)
test_end_date = analysis_end

all_dates = pd.to_datetime(master_df['Date'])
master_df = master_df[(all_dates >= analysis_start) & (all_dates <= analysis_end)].copy()
all_dates = pd.to_datetime(master_df['Date'])

train_mask = all_dates <= train_end_date
test_mask = (all_dates >= test_start_date) & (all_dates <= test_end_date)

train_df = master_df[train_mask].copy()
test_df = master_df[test_mask].copy()

print('Analysis range:', analysis_start.date(), '→', analysis_end.date())
print('Train split end:', train_end_date.date(), '| Test start:', test_start_date.date())
print('Train:', train_df['Date'].min(), '→', train_df['Date'].max(), f'({len(train_df):,} rows)')
print('Test :', test_df['Date'].min(), '→', test_df['Date'].max(), f'({len(test_df):,} rows)')





In [ ]:
# ============================================================================
# NORMALIZATION (TRAIN-FIT, TEST-TRANSFORM)
# ============================================================================
from collections import Counter

feature_cols = processor.get_feature_columns('phase1')

master_df_normalized, scalers = processor.normalize_features(
    master_df,
    feature_cols=feature_cols,
    train_end_date=train_end_date,
    test_start_date=test_start_date,
    existing_scalers=None,
    scaler_type='standard',
)

actuarial_cols = [c for c in master_df_normalized.columns if c.startswith('Actuarial_')]

method_counter = Counter()
for c in feature_cols:
    spec = scalers.get(c)
    if isinstance(spec, dict):
        method_counter[str(spec.get('method', 'standard'))] += 1
    elif spec is None:
        method_counter['missing'] += 1
    else:
        method_counter['legacy_scaler'] += 1

print('✅ Normalization complete')
print('Normalized shape:', master_df_normalized.shape)
print('Actuarial columns:', actuarial_cols)
print('Normalization strategy counts:', dict(method_counter))


In [ ]:
# ============================================================================
# BUILD PHASE1 DATASET CONTAINER
# ============================================================================
all_dates_norm = pd.to_datetime(master_df_normalized['Date'])
train_df_norm = master_df_normalized[all_dates_norm <= train_end_date].copy()
test_df_norm = master_df_normalized[(all_dates_norm >= test_start_date) & (all_dates_norm <= test_end_date)].copy()

covariance_columns = identify_covariance_columns(master_df_normalized.columns)

phase1_data = Phase1Dataset(
    master_df=master_df_normalized,
    train_df=train_df_norm,
    test_df=test_df_norm,
    scalers=scalers,
    train_end_date=train_end_date,
    test_start_date=test_start_date,
    covariance_columns=covariance_columns,
    data_processor=processor,
)

print('✅ Phase1Dataset ready')
print('Train shape:', phase1_data.train_df.shape)
print('Test shape :', phase1_data.test_df.shape)
print('Covariance features:', len(covariance_columns))
print('Assets:', sorted(phase1_data.master_df['Ticker'].dropna().unique().tolist()))


## 4) Training


In [ ]:
# Optional stability tweaks (keep commented unless needed)
# config['training_params']['timesteps_per_ppo_update'] = 504
# ppo = config['agent_params']['ppo_params']
# ppo['batch_size_ppo'] = 512
# ppo['num_ppo_epochs'] = 5
# ppo['actor_lr'] = 5e-5
# ppo['critic_lr'] = 1e-4
# ppo['policy_clip'] = 0.15
# ppo['target_kl'] = 0.02
# ppo['entropy_coef'] = 0.01
# config['environment_params']['penalty_budget_ratio'] = 1.25
# config['environment_params']['intra_step_tape_delta_beta'] = 0.10
# config['environment_params']['intra_step_tape_delta_clip'] = 0.20


In [ ]:
# ============================================================================
# TRAIN ACTIVE VARIANT
# ============================================================================
RUN_TRAINING = True

if RUN_TRAINING:
    train_cfg = config['training_params']
    print('🚀 Starting training')
    print('Variant:', ACTIVE_VARIANT)
    print('max_total_timesteps:', train_cfg['max_total_timesteps'])
    print('timesteps_per_ppo_update:', train_cfg['timesteps_per_ppo_update'])

    experiment6 = run_experiment6_tape(
        phase1_data=phase1_data,
        config=config,
        random_seed=RANDOM_SEED,
        csv_logger_cls=CSVLogger,
        use_covariance=True,
        architecture=config['agent_params']['actor_critic_type'],
        timesteps_per_update=train_cfg['timesteps_per_ppo_update'],
        max_total_timesteps=train_cfg['max_total_timesteps'],
    )

    print('✅ Training complete')
    print('checkpoint_prefix:', experiment6.checkpoint_path)
else:
    print('ℹ️ RUN_TRAINING=False (set True to train)')


In [ ]:
# ============================================================================
# QUICK TRAINING LOG INSPECTION (LATEST)
# ============================================================================
logs_dir = Path(LATEST_RESULTS_ROOT) / 'logs'
logs_dir.mkdir(parents=True, exist_ok=True)

episodes_files = sorted(logs_dir.glob('*episodes*.csv'), key=lambda p: p.stat().st_mtime, reverse=True)
if not episodes_files:
    print(f'No episodes CSV found in {logs_dir} yet.')
else:
    epis_path = episodes_files[0]
    episodes_df = pd.read_csv(epis_path)
    print('Episodes file:', epis_path)
    print('Rows:', len(episodes_df))
    display(episodes_df.head())


## 5) Evaluation (Unified Multi-Track)


In [ ]:
# ============================================================================
# OPTIONAL: RELOAD TRAINING METADATA (POST-RESTART)
# ============================================================================
USE_METADATA_RELOAD = False
METADATA_PATH = None  # e.g., Path('tcn_results/logs/Exp6_TCN_Enhanced_TAPE_training_YYYYMMDD_HHMMSS_metadata.json')

if USE_METADATA_RELOAD:
    if METADATA_PATH is None:
        logs_dir = Path(LATEST_RESULTS_ROOT) / 'logs'
        cand = sorted(logs_dir.glob('*metadata*.json'), key=lambda p: p.stat().st_mtime, reverse=True)
        METADATA_PATH = cand[0] if cand else None

    if METADATA_PATH and Path(METADATA_PATH).exists():
        config = load_training_metadata_into_config(Path(METADATA_PATH), config, verbose=True)
        print('✅ Metadata reloaded from:', METADATA_PATH)
    else:
        print('⚠️ Metadata file not found; continuing with current config.')
else:
    print('ℹ️ USE_METADATA_RELOAD=False')


In [ ]:
# ============================================================================
# UNIFIED EVALUATION: det_mode + det_mean + stochastic
# ============================================================================
RUN_EVAL = True

# Model selection
MODEL_FAMILY = 'normal'           # normal | rare | clip
NORMAL_MODEL_STRATEGY = 'latest'  # latest | final
RARE_MODEL_STRATEGY = 'best'      # best | episode
CHECKPOINT_EPISODE = 83           # used when rare_model_strategy='episode'
CLIP_EPISODE = 83                 # used when model_family='clip'
CHECKPOINT_PREFIX_OVERRIDE = None # e.g., 'tcn_fusion_results/exp6_tape_ep83'

if RUN_EVAL:
    experiment6_stub = create_experiment6_result_stub(
        random_seed=RANDOM_SEED,
        use_covariance=True,
        architecture=config['agent_params']['actor_critic_type'],
        checkpoint_path=None,
        base_agent_params=config.get('agent_params'),
    )

    evaluation_stub = evaluate_experiment6_checkpoint(
        experiment6_stub,
        phase1_data=phase1_data,
        config=config,
        random_seed=RANDOM_SEED,
        model_family=MODEL_FAMILY,
        normal_model_strategy=NORMAL_MODEL_STRATEGY,
        rare_model_strategy=RARE_MODEL_STRATEGY,
        checkpoint_episode=CHECKPOINT_EPISODE,
        clip_episode=CLIP_EPISODE,
        checkpoint_path_override=CHECKPOINT_PREFIX_OVERRIDE,
        num_eval_runs=30,
        compare_deterministic_modes=['mode', 'mean'],
        stochastic_eval_mode='sample',
        sample_actions_stochastic=True,
        sample_actions=None,
        stochastic_episode_length_limit=252,
        save_eval_logs=True,
        save_eval_artifacts=True,
    )

    print('✅ Evaluation complete')
    print('Checkpoint:', evaluation_stub.actor_weights_path)
    print('Eval CSV  :', evaluation_stub.eval_results_path)
else:
    print('ℹ️ RUN_EVAL=False (set True to evaluate)')


In [ ]:
# ============================================================================
# EVALUATION ARTIFACT EXPORTS (VARIANT-SCOPED)
# ============================================================================
from datetime import datetime

if 'evaluation_stub' not in globals():
    print('Run evaluation first (RUN_EVAL=True).')
else:
    assets = ASSET_TICKERS + ['Cash']

    results_root = Path(globals().get('LATEST_RESULTS_ROOT', 'tcn_results'))
    stamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    out_root = results_root / 'model_outputs' / f'eval_{stamp}'
    det_out = out_root / 'deterministic'
    sto_out = out_root / 'stochastic'
    det_out.mkdir(parents=True, exist_ok=True)
    sto_out.mkdir(parents=True, exist_ok=True)

    # Deterministic artifacts
    det_dates = pd.DatetimeIndex(evaluation_stub.env_test_deterministic.dates)

    if len(evaluation_stub.deterministic_alphas) > 0:
        df_alpha = pd.DataFrame(evaluation_stub.deterministic_alphas, columns=assets)
        df_alpha.index = det_dates[:len(df_alpha)]
        df_alpha.index.name = 'date'
        df_alpha.to_csv(det_out / 'alphas.csv')

    if len(evaluation_stub.deterministic_weights) > 0:
        df_w = pd.DataFrame(evaluation_stub.deterministic_weights, columns=assets)
        df_w.index = det_dates[:len(df_w)]
        df_w.index.name = 'date'
        df_w.to_csv(det_out / 'weights.csv')

    if len(evaluation_stub.deterministic_actions) > 0:
        df_a = pd.DataFrame(evaluation_stub.deterministic_actions, columns=assets)
        df_a.index = det_dates[:len(df_a)]
        df_a.index.name = 'date'
        df_a.to_csv(det_out / 'actions.csv')

    # Copy eval summary CSV into output root for traceability
    eval_csv_path = Path(evaluation_stub.eval_results_path) if evaluation_stub.eval_results_path else None
    if eval_csv_path and eval_csv_path.exists():
        df_eval = pd.read_csv(eval_csv_path)
        df_eval.to_csv(out_root / 'evaluation_summary.csv', index=False)
    else:
        df_eval = pd.DataFrame()

    # Stochastic artifacts
    all_dates = pd.DatetimeIndex(evaluation_stub.env_test_random.dates)
    actions_rows, weights_rows, alphas_rows = [], [], []

    if isinstance(evaluation_stub.stochastic_results, pd.DataFrame) and not evaluation_stub.stochastic_results.empty:
        stochastic_results_df = evaluation_stub.stochastic_results.copy()
        stochastic_results_df.to_csv(sto_out / 'stochastic_results.csv', index=False)

        for i in range(len(stochastic_results_df)):
            run_id = int(stochastic_results_df.iloc[i].get('run', i + 1))
            start_date = pd.Timestamp(stochastic_results_df.iloc[i]['start_date'])
            start_idx = all_dates.get_loc(start_date)

            run_actions = evaluation_stub.stochastic_actions[i] if i < len(evaluation_stub.stochastic_actions) else []
            run_weights = evaluation_stub.stochastic_weights[i] if i < len(evaluation_stub.stochastic_weights) else []
            run_alphas = evaluation_stub.stochastic_alphas[i] if i < len(evaluation_stub.stochastic_alphas) else []

            run_dates = all_dates[start_idx:start_idx + len(run_weights)]

            if len(run_actions):
                dfa = pd.DataFrame(run_actions, columns=assets)
                dfa['run'] = run_id
                dfa['date'] = run_dates[:len(dfa)]
                actions_rows.append(dfa)

            if len(run_weights):
                dfw = pd.DataFrame(run_weights, columns=assets)
                dfw['run'] = run_id
                dfw['date'] = run_dates[:len(dfw)]
                weights_rows.append(dfw)

            if len(run_alphas):
                dfl = pd.DataFrame(run_alphas, columns=assets)
                dfl['run'] = run_id
                dfl['date'] = run_dates[:len(dfl)]
                alphas_rows.append(dfl)

    if actions_rows:
        pd.concat(actions_rows, ignore_index=True).set_index(['run', 'date']).to_csv(sto_out / 'actions_all_runs.csv')
    if weights_rows:
        pd.concat(weights_rows, ignore_index=True).set_index(['run', 'date']).to_csv(sto_out / 'weights_all_runs.csv')
    if alphas_rows:
        pd.concat(alphas_rows, ignore_index=True).set_index(['run', 'date']).to_csv(sto_out / 'alphas_all_runs.csv')

    # README with current run context
    readme_lines = [
        '# Evaluation Artifact Export',
        '',
        f'- Variant results root: `{results_root}`',
        f'- Export root: `{out_root}`',
        f'- Checkpoint actor: `{evaluation_stub.actor_weights_path}`',
        f'- Checkpoint critic: `{evaluation_stub.critic_weights_path}`',
        f'- Eval summary CSV: `{evaluation_stub.eval_results_path}`',
        f'- Export timestamp: `{stamp}`',
        '',
        '## Included Files',
        '- `deterministic/weights.csv`',
        '- `deterministic/actions.csv`',
        '- `deterministic/alphas.csv`',
        '- `stochastic/stochastic_results.csv` (if stochastic runs were executed)',
        '- `stochastic/weights_all_runs.csv`',
        '- `stochastic/actions_all_runs.csv`',
        '- `stochastic/alphas_all_runs.csv`',
        '- `evaluation_summary.csv`',
    ]

    if not df_eval.empty:
        cols = [
            'eval_track', 'evaluation_type', 'start_date', 'market_regime',
            'mean_concentration_hhi', 'mean_top_weight',
            'mean_action_realization_l1', 'max_action_realization_l1'
        ]
        present = [c for c in cols if c in df_eval.columns]
        readme_lines += ['', '## Key Logged Diagnostics (present in summary CSV)', *(f'- `{c}`' for c in present)]

    (out_root / 'README.md').write_text('\n'.join(readme_lines), encoding='utf-8')

    print('✅ Export complete')
    print('Export root:', out_root)
    print('Deterministic dir:', det_out)
    print('Stochastic dir   :', sto_out)


In [ ]:
# ============================================================================
# EVAL CSV DIAGNOSTIC COLUMN CHECK
# ============================================================================
required_cols = [
    'start_date',
    'market_regime',
    'mean_concentration_hhi',
    'mean_top_weight',
    'mean_action_realization_l1',
    'max_action_realization_l1',
]

csv_path = None

if 'evaluation_stub' in globals() and getattr(evaluation_stub, 'eval_results_path', None):
    p = Path(evaluation_stub.eval_results_path)
    if p.exists():
        csv_path = p

if csv_path is None:
    root = Path(globals().get('LATEST_RESULTS_ROOT', 'tcn_results'))
    logs_dir = root / 'logs'
    candidates = sorted(logs_dir.glob('*_eval_*.csv'), key=lambda x: x.stat().st_mtime, reverse=True) if logs_dir.exists() else []
    csv_path = candidates[0] if candidates else None

if csv_path is None:
    print('⚠️ No evaluation CSV found. Run evaluation first.')
else:
    df_eval = pd.read_csv(csv_path)
    present = [c for c in required_cols if c in df_eval.columns]
    missing = [c for c in required_cols if c not in df_eval.columns]

    print('📂 Eval CSV:', csv_path)
    print('Rows:', len(df_eval))
    print('Required columns present:', len(present), '/', len(required_cols))

    if missing:
        print('❌ Missing columns:', missing)
    else:
        print('✅ All required diagnostic columns are present.')

    show_cols = ['eval_track', 'evaluation_type'] + [c for c in required_cols if c in df_eval.columns]
    show_cols = [c for c in show_cols if c in df_eval.columns]
    if show_cols:
        display(df_eval[show_cols].head(10))


In [ ]:
# ============================================================================
# DIAGNOSTICS SUMMARY
# ============================================================================
if 'evaluation_stub' not in globals():
    print('Run evaluation first (RUN_EVAL=True).')
else:
    # stochastic summary
    stoch = evaluation_stub.stochastic_results.copy()
    if stoch is not None and not stoch.empty:
        cols = [
            'total_return', 'annualized_return', 'sharpe_ratio', 'sortino_ratio',
            'max_drawdown', 'volatility', 'turnover', 'win_rate'
        ]
        cols = [c for c in cols if c in stoch.columns]
        print('Stochastic summary:')
        display(stoch[cols].describe().T)

    # deterministic diagnostics
    acts = np.asarray(evaluation_stub.deterministic_actions)
    alps = np.asarray(evaluation_stub.deterministic_alphas)

    action_uniques = int(np.unique(np.round(acts, 6), axis=0).shape[0]) if acts.size else 0
    alpha_le1_frac = float(np.mean(alps <= 1.0)) if alps.size else 0.0
    argmax_uniques = int(np.unique(np.argmax(alps, axis=1)).shape[0]) if (alps.ndim == 2 and len(alps) > 0) else 0

    print('Deterministic diagnostics:')
    print(' action_uniques      =', action_uniques)
    print(' alpha<=1 fraction   =', alpha_le1_frac)
    print(' argmax_alpha_uniques=', argmax_uniques)


In [ ]:
# ============================================================================
# FIXED STRESS-WINDOW EVALUATION (2 WINDOWS)
# ============================================================================
import pandas as pd
from dataclasses import replace

STRESS_WINDOWS = [
    ('2020-02-20', '2020-05-29', 'COVID crash + rebound'),
    ('2022-01-03', '2022-12-30', 'Rate-hike bear year'),
]

def subset_phase1_test_window(phase1_data, start_date, end_date):
    s = pd.Timestamp(start_date)
    e = pd.Timestamp(end_date)
    df = phase1_data.test_df.copy()
    df['Date'] = pd.to_datetime(df['Date'])
    win = df[(df['Date'] >= s) & (df['Date'] <= e)].copy()
    if win.empty:
        raise ValueError(f'No rows in test_df for {start_date} -> {end_date}')
    return replace(
        phase1_data,
        test_df=win,
        test_start_date=win['Date'].min(),
        train_end_date=win['Date'].max(),
    )

fixed_rows = []
for start, end, label in STRESS_WINDOWS:
    phase_win = subset_phase1_test_window(phase1_data, start, end)

    ev = evaluate_experiment6_checkpoint(
        experiment6_stub,
        phase1_data=phase_win,
        config=config,
        random_seed=RANDOM_SEED,
        checkpoint_path_override=CHECKPOINT_PREFIX_OVERRIDE,
        deterministic_eval_mode='mode',
        num_eval_runs=0,
        stochastic_eval_mode='sample',
        save_eval_logs=False,
        save_eval_artifacts=False,
    )

    m = ev.deterministic_metrics or {}
    fixed_rows.append({
        'window_label': label,
        'start': start,
        'end': end,
        'days_traded': len(ev.deterministic_portfolio) - 1 if len(ev.deterministic_portfolio) else 0,
        'total_return': m.get('total_return'),
        'annualized_return': m.get('annualized_return'),
        'sharpe': m.get('sharpe_ratio'),
        'sortino': m.get('sortino_ratio'),
        'max_drawdown': m.get('max_drawdown_abs', m.get('max_drawdown')),
        'volatility': m.get('volatility'),
        'turnover': m.get('turnover'),
        'win_rate': m.get('win_rate'),
    })

fixed_df = pd.DataFrame(fixed_rows)
display(fixed_df.sort_values('start'))


## 6) Checkpoint Scan


In [ ]:
# ============================================================================
# CHECKPOINT SCANNER (DETERMINISTIC)
# ============================================================================
import re


def evaluate_checkpoint_range_deterministic(
    episode_range=(2, 300),
    results_root=None,
    random_seed=RANDOM_SEED,
    deterministic_eval_mode='mode',
):
    low, high = episode_range
    base_root = Path(results_root) if results_root else Path(LATEST_RESULTS_ROOT)

    checkpoints = {}
    for root in [base_root, base_root / 'rare_models']:
        if not root.exists():
            continue
        for actor_path in root.glob('*_actor.weights.h5'):
            m = re.search(r'_ep(\d+)', actor_path.name)
            if not m:
                continue
            ep = int(m.group(1))
            if low <= ep <= high:
                checkpoints[ep] = actor_path

    if not checkpoints:
        print(f'No checkpoints found in {base_root} for range {episode_range}.')
        return None

    rows = []
    for ep, actor_path in sorted(checkpoints.items()):
        prefix = str(actor_path).replace('_actor.weights.h5', '')

        stub = create_experiment6_result_stub(
            random_seed=random_seed,
            use_covariance=True,
            architecture=config['agent_params']['actor_critic_type'],
            checkpoint_path=prefix,
            base_agent_params=config.get('agent_params'),
        )

        ev = evaluate_experiment6_checkpoint(
            experiment6=stub,
            phase1_data=phase1_data,
            config=config,
            random_seed=random_seed,
            checkpoint_path_override=prefix,
            model_family='normal',
            normal_model_strategy='latest',
            num_eval_runs=0,
            deterministic_eval_mode=deterministic_eval_mode,
            save_eval_logs=False,
            save_eval_artifacts=False,
        )

        m = ev.deterministic_metrics or {}
        rows.append({
            'episode': ep,
            'checkpoint_prefix': prefix,
            'sharpe': m.get('sharpe_ratio', float('nan')),
            'total_return': m.get('total_return', float('nan')),
            'max_drawdown': m.get('max_drawdown_abs', m.get('max_drawdown', float('nan'))),
            'turnover': m.get('turnover', float('nan')),
        })

    df_scores = pd.DataFrame(rows).sort_values('sharpe', ascending=False)
    return df_scores


In [ ]:
# ============================================================================
# RUN CHECKPOINT SCAN
# ============================================================================
RUN_SCAN = True

if RUN_SCAN:
    results_root = Path(globals().get('LATEST_RESULTS_ROOT', 'tcn_results'))
    print('Scanning:', results_root)
    scan_df = evaluate_checkpoint_range_deterministic(
        episode_range=(8, 100),
        results_root=results_root,
        random_seed=RANDOM_SEED,
        deterministic_eval_mode='mean',
    )
    display(scan_df.head(20) if scan_df is not None else None)

    step_dir = results_root / 'step_sharpe_checkpoints'
    if step_dir.exists():
        step_files = sorted(step_dir.glob('*_actor.weights.h5'))
        print(f'Step-Sharpe checkpoints found: {len(step_files)}')
        for p in step_files[:10]:
            print(' -', p.name)
    else:
        print('No step_sharpe_checkpoints directory yet.')
else:
    print('ℹ️ RUN_SCAN=False')


## 7) Overfit Monitor (Train-Test Gap)


In [ ]:
# ============================================================================
# OVERFIT MONITOR HELPERS
# ============================================================================
import re
from dataclasses import replace


def _infer_results_root_for_notebook(cfg):
    arch = cfg.get('agent_params', {}).get('actor_critic_type', 'TCN').upper()
    use_attention = bool(cfg.get('agent_params', {}).get('use_attention', False))
    use_fusion = bool(cfg.get('agent_params', {}).get('use_fusion', False))
    if arch.startswith('TCN'):
        if use_fusion:
            return Path('tcn_fusion_results')
        if use_attention:
            return Path('tcn_att_results')
        return Path('tcn_results')
    return Path('tcn_results')


def _discover_checkpoint_prefixes(results_root, episode_range=(1, 9999), include_rare=True):
    lo, hi = episode_range
    roots = [Path(results_root)]
    if include_rare:
        roots.append(Path(results_root) / 'rare_models')

    prefixes = {}
    for root in roots:
        if not root.exists():
            continue
        for actor in root.glob('*_actor.weights.h5'):
            m = re.search(r'_ep(\d+)', actor.name)
            if not m:
                continue
            ep = int(m.group(1))
            if lo <= ep <= hi:
                prefixes[ep] = str(actor).replace('_actor.weights.h5', '')

    return [(ep, prefixes[ep]) for ep in sorted(prefixes.keys())]


def _subset_phase1_for_eval(phase1_data, split='test'):
    split = str(split).lower().strip()
    if split not in {'train', 'test'}:
        raise ValueError(f'split must be train or test, got: {split}')

    eval_df = phase1_data.train_df.copy() if split == 'train' else phase1_data.test_df.copy()
    start_date = pd.to_datetime(eval_df['Date']).min()
    end_date = pd.to_datetime(eval_df['Date']).max()

    return replace(
        phase1_data,
        test_df=eval_df,
        test_start_date=start_date,
        train_end_date=end_date,
    )


def _diagnostics_from_eval(ev):
    acts = np.asarray(ev.deterministic_actions)
    alps = np.asarray(ev.deterministic_alphas)
    action_uniques = int(np.unique(np.round(acts, 6), axis=0).shape[0]) if acts.size else 0
    alpha_le1_fraction = float(np.mean(alps <= 1.0)) if alps.size else 0.0
    argmax_alpha_uniques = int(np.unique(np.argmax(alps, axis=1)).shape[0]) if (alps.ndim == 2 and len(alps) > 0) else 0
    return action_uniques, alpha_le1_fraction, argmax_alpha_uniques


def run_checkpoint_overfit_monitor(
    phase1_data,
    config,
    random_seed,
    episode_range=(1, 300),
    deterministic_modes=('mode', 'mean'),
    eval_splits=('train', 'test'),
    results_root=None,
    include_rare=False,
    save_csv=True,
):
    results_root = Path(results_root) if results_root else _infer_results_root_for_notebook(config)
    ckpts = _discover_checkpoint_prefixes(results_root, episode_range=episode_range, include_rare=include_rare)
    if not ckpts:
        raise RuntimeError(f'No checkpoints found in {results_root} for range {episode_range}.')

    if isinstance(deterministic_modes, str):
        deterministic_modes = (deterministic_modes,)
    if isinstance(eval_splits, str):
        eval_splits = (eval_splits,)

    eval_splits = tuple(str(s).lower().strip() for s in eval_splits)
    bad = [s for s in eval_splits if s not in {'train', 'test'}]
    if bad:
        raise ValueError(f'Invalid eval_splits entries: {bad}. Allowed: train, test')

    stub = create_experiment6_result_stub(
        random_seed=random_seed,
        use_covariance=True,
        architecture=config['agent_params']['actor_critic_type'],
        checkpoint_path=ckpts[0][1],
        base_agent_params=config.get('agent_params'),
    )

    rows = []
    for ep, prefix in ckpts:
        for split in eval_splits:
            phase_eval = _subset_phase1_for_eval(phase1_data, split=split)
            split_start = pd.to_datetime(phase_eval.test_df['Date']).min()
            split_end = pd.to_datetime(phase_eval.test_df['Date']).max()

            for mode in deterministic_modes:
                ev = evaluate_experiment6_checkpoint(
                    stub,
                    phase1_data=phase_eval,
                    config=config,
                    random_seed=random_seed,
                    checkpoint_path_override=prefix,
                    deterministic_eval_mode=mode,
                    num_eval_runs=0,
                    stochastic_eval_mode='sample',
                    save_eval_logs=False,
                    save_eval_artifacts=False,
                )

                m = ev.deterministic_metrics or {}
                action_uniques, alpha_le1_fraction, argmax_alpha_uniques = _diagnostics_from_eval(ev)

                rows.append({
                    'checkpoint_prefix': prefix,
                    'episode': ep,
                    'architecture': config['agent_params']['actor_critic_type'],
                    'split': split,
                    'deterministic_mode': mode,
                    'seed': random_seed,
                    'window_start': split_start,
                    'window_end': split_end,
                    'days_traded': int(len(ev.deterministic_portfolio) - 1) if len(ev.deterministic_portfolio) else 0,
                    'total_return': float(m.get('total_return', np.nan)),
                    'annualized_return': float(m.get('annualized_return', np.nan)),
                    'sharpe_ratio': float(m.get('sharpe_ratio', np.nan)),
                    'sortino_ratio': float(m.get('sortino_ratio', np.nan)),
                    'max_drawdown': float(m.get('max_drawdown_abs', m.get('max_drawdown', np.nan))),
                    'volatility': float(m.get('volatility', np.nan)),
                    'turnover': float(m.get('turnover', np.nan)),
                    'win_rate': float(m.get('win_rate', np.nan)),
                    'action_uniques': action_uniques,
                    'alpha_le1_fraction': alpha_le1_fraction,
                    'argmax_alpha_uniques': argmax_alpha_uniques,
                })

    df = pd.DataFrame(rows)
    if df.empty:
        raise RuntimeError('Monitor produced no rows.')

    left = df[df['split'] == 'train'].copy()
    right = df[df['split'] == 'test'].copy()
    if left.empty or right.empty:
        raise RuntimeError("Overfit summary requires BOTH train and test rows. Use eval_splits=('train','test').")

    keys = ['checkpoint_prefix', 'episode', 'architecture', 'deterministic_mode', 'seed']
    summary = left.merge(right, on=keys, suffixes=('_train', '_test'))

    summary['sharpe_gap'] = summary['sharpe_ratio_train'] - summary['sharpe_ratio_test']
    summary['mdd_gap'] = summary['max_drawdown_test'] - summary['max_drawdown_train']
    summary['return_gap'] = summary['annualized_return_train'] - summary['annualized_return_test']

    summary['flag_overfit'] = (
        (summary['sharpe_gap'] > 0.40)
        | (summary['mdd_gap'] > 0.05)
        | (summary['return_gap'] > 0.10)
    )

    summary = summary.sort_values(['flag_overfit', 'sharpe_ratio_test'], ascending=[True, False]).reset_index(drop=True)

    out_path = None
    if save_csv:
        out_dir = Path(results_root) / 'logs'
        out_dir.mkdir(parents=True, exist_ok=True)
        ts = datetime.now().strftime('%Y%m%d_%H%M%S')
        out_path = out_dir / f'checkpoint_overfit_monitor_{ts}.csv'
        summary.to_csv(out_path, index=False)
        print('💾 Overfit monitor saved:', out_path)

    return df, summary, out_path


In [ ]:
# ============================================================================
# RUN OVERFIT MONITOR
# ============================================================================
RUN_OVERFIT_MONITOR = True

if RUN_OVERFIT_MONITOR:
    results_root = Path(globals().get('LATEST_RESULTS_ROOT', _infer_results_root_for_notebook(config)))
    print('Using results root:', results_root)

    monitor_rows_df, monitor_summary_df, monitor_csv_path = run_checkpoint_overfit_monitor(
        phase1_data=phase1_data,
        config=config,
        random_seed=RANDOM_SEED,
        episode_range=(79, 100),
        deterministic_modes=('mode', 'mean'),
        eval_splits=('train', 'test'),
        results_root=results_root,
        include_rare=True,
        save_csv=True,
    )

    display(monitor_summary_df.head(20))

    if not monitor_summary_df.empty:
        best = (
            monitor_summary_df[monitor_summary_df['flag_overfit'] == False]
            .sort_values('sharpe_ratio_test', ascending=False)
            .head(10)
        )
        print('Top non-overfit candidates (by test Sharpe):')
        display(best[[
            'episode', 'deterministic_mode', 'sharpe_ratio_test',
            'max_drawdown_test', 'turnover_test', 'sharpe_gap', 'mdd_gap', 'return_gap'
        ]])
else:
    print('ℹ️ RUN_OVERFIT_MONITOR=False')


## 8) Optional Analysis Utilities


In [ ]:
# ============================================================================
# OPTIONAL: ABLATION TABLE + TRACK SUMMARY
# ============================================================================
RUN_OPTIONAL_ANALYSIS = False

if RUN_OPTIONAL_ANALYSIS:
    try:
        from src.notebook_helpers.tcn_phase1 import build_ablation_table, build_evaluation_track_summary
    except Exception as e:
        raise RuntimeError(f'Optional analysis helpers unavailable: {e}')

    available = {k: v for k, v in globals().items() if k.startswith('evaluation_') and hasattr(v, 'deterministic_metrics')}
    if 'evaluation_stub' in globals():
        available.setdefault('current_eval', evaluation_stub)

    if not available:
        print('No evaluation objects found. Run evaluation first.')
    else:
        display(build_ablation_table(available))
        if 'evaluation_stub' in globals():
            print('Track summary for current evaluation:')
            display(build_evaluation_track_summary(evaluation_stub))
else:
    print('ℹ️ RUN_OPTIONAL_ANALYSIS=False')


## 9) Run Checklist

Before running heavy jobs:
- Confirm `ACTIVE_VARIANT`
- Confirm `max_total_timesteps` and `timesteps_per_ppo_update`
- Confirm curriculum schedule (1500 -> 2000 -> 2500 -> full)
- Confirm step-Sharpe checkpoint rule (`>= 0.5`)
- Confirm intra-step TAPE delta settings (`beta`, `clip`, `window`)
- Set exactly one expensive toggle at a time (`RUN_TRAINING`, `RUN_EVAL`, `RUN_SCAN`, `RUN_OVERFIT_MONITOR`)
- Keep artifact exports on after successful eval


## 10) Feature Manifest Audit (Latest Artifacts)

Use this section to inspect the latest active-feature manifests without hardcoded timestamps.


In [ ]:
import json
from pathlib import Path
from src.notebook_helpers.tcn_phase1 import summarize_active_feature_manifest

def latest_path(paths):
    paths = [p for p in paths if p.exists()]
    if not paths:
        return None
    return sorted(paths, key=lambda p: p.stat().st_mtime, reverse=True)[0]

trainrl_manifest = latest_path(Path('results').glob('**/active_feature_manifest.json'))
notebook_manifest = latest_path(Path(LATEST_RESULTS_ROOT).glob('logs/*_active_feature_manifest.json'))

print('Latest train_rl manifest:', trainrl_manifest)
print('Latest notebook manifest:', notebook_manifest)

if trainrl_manifest:
    summarize_active_feature_manifest(str(trainrl_manifest))
if notebook_manifest:
    summarize_active_feature_manifest(str(notebook_manifest))


In [ ]:
# Inspect missing requested columns + group counts from latest notebook manifest
manifest_path = notebook_manifest
if manifest_path is None:
    print('No notebook manifest found. Run training first.')
else:
    m = json.loads(Path(manifest_path).read_text(encoding='utf-8'))
    train_env = m.get('train_env', m)
    print('Manifest path:', manifest_path)
    print('Missing requested columns:', train_env.get('missing_requested_columns', []))
    print('Group counts:', train_env.get('group_counts', {}))


In [ ]:
# Print active variable names from latest notebook manifest
manifest_path = notebook_manifest
if manifest_path is None:
    print('No notebook manifest found. Run training first.')
else:
    m = json.loads(Path(manifest_path).read_text(encoding='utf-8'))
    train_env = m.get('train_env', m)
    active = train_env.get('active_feature_columns', [])
    print(f'Active variable count: {len(active)}')
    for v in active:
        print(v)
